In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("churn.csv", encoding="latin1")
df = pd.read_csv("churn.csv", sep=",", engine="python")

In [4]:
df = df.drop(columns=["RowNumber","CustomerId","Surname"])
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df = pd.get_dummies(df, columns=["Geography","Gender"], drop_first=False)
df.head()


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True,True,False


In [6]:
from sklearn.model_selection import train_test_split

X = df.drop("Exited", axis=1)
y = df["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=300, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=300, random_state=42)

In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))


Accuracy: 0.8655
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.76      0.46      0.58       393

    accuracy                           0.87      2000
   macro avg       0.82      0.71      0.75      2000
weighted avg       0.86      0.87      0.85      2000

[[1549   58]
 [ 211  182]]


In [9]:
df.to_csv("bank_clean.csv", index=False)


In [10]:
import joblib
joblib.dump(model, "clean_churn_model.pkl")


['clean_churn_model.pkl']

In [11]:
predictions = model.predict(X)
prob = model.predict_proba(X)[:,1]

df_demo = df.copy()
df_demo["predicted_churn"] = predictions
df_demo["churn_probability"] = prob

df_demo.to_csv("predictions.csv", index=False)
